In [2]:
import pandas as pd
import gc
import os
from tqdm import tqdm

# ▶ Base 경로
base_path = "마이박스 다운로드\\최종EDA_unique제거 전체\\test\\"

# ▶ 병합 대상 파일 목록
file_names = [
    "test_1_회원정보_모델링용.csv",
    "test_2_신용정보_모델링용.csv",
    "test_3_매출정보_unique_학습용.csv",
    "test_4.청구입금정보_변동없음.csv",
    "test_5.잔액정보_EDA.csv",
    "test_6.채널정보_unique.csv",
    "test_7_마케팅정보_unique.csv",
    "test_8_성과정보_변동없음_결측치처리.csv"
]
file_paths = [base_path + fname for fname in file_names]

# ▶ 데이터 타입 최적화 함수
def optimize_types(df):
    for col in df.select_dtypes(include='number').columns:
        if 'float' in str(df[col].dtype):
            df[col] = pd.to_numeric(df[col], downcast='float')
        elif 'int' in str(df[col].dtype):
            df[col] = pd.to_numeric(df[col], downcast='integer')
    for col in df.select_dtypes(include='object').columns:
        if df[col].nunique() / len(df) < 0.5:
            df[col] = df[col].astype('category')
    return df

# ▶ 저장 경로
step_path = "병합/merged_step_test.csv"
final_path = "병합/전체 병합_test.csv"

# ▶ 재시작 여부 판단
if os.path.exists(step_path):
    print("✅ 이전 병합 파일을 불러와 이어서 진행합니다.")
    merged = pd.read_csv(step_path)
    start_idx = len(merged.columns) - 1  # ID 포함
else:
    print("🆕 새로운 병합을 시작합니다.")
    df = pd.read_csv(file_paths[0])
    df = optimize_types(df)
    merged = df.copy()
    merged.to_csv(step_path, index=False)
    print(f"✅ 첫 파일 저장 완료: {step_path}")
    del df
    gc.collect()
    start_idx = 1  # 첫 파일 이후부터 시작

# ▶ tqdm으로 병합 진행
for i in tqdm(range(start_idx, len(file_paths)), desc="📁 파일 병합 중", unit="file"):
    file = file_paths[i]
    print(f"\n📄 병합 중: {file_names[i]}")
    df = pd.read_csv(file)
    df = optimize_types(df)

    merged = pd.concat([merged, df.drop(columns=['ID'], errors='ignore')], axis=1)
    merged.to_csv(step_path, index=False)
    print(f"✅ 중간 저장 완료: {step_path} (열 {merged.shape[1]})")

    del df
    gc.collect()

# ▶ 중복 컬럼 제거 (ID 등)
merged = merged.loc[:, ~merged.columns.duplicated()]

# ▶ 최종 저장
merged.to_csv(final_path, index=False)
print(f"\n🎉 최종 병합 완료: {final_path} (행 {merged.shape[0]}, 열 {merged.shape[1]})")


🆕 새로운 병합을 시작합니다.
✅ 첫 파일 저장 완료: 병합/merged_step_test.csv


📁 파일 병합 중:   0%|                                                                         | 0/7 [00:00<?, ?file/s]


📄 병합 중: test_2_신용정보_모델링용.csv


📁 파일 병합 중:  14%|█████████▎                                                       | 1/7 [00:12<01:12, 12.15s/file]

✅ 중간 저장 완료: 병합/merged_step_test.csv (열 104)

📄 병합 중: test_3_매출정보_unique_학습용.csv


📁 파일 병합 중:  29%|██████████████████▌                                              | 2/7 [01:03<02:55, 35.15s/file]

✅ 중간 저장 완료: 병합/merged_step_test.csv (열 501)

📄 병합 중: test_4.청구입금정보_변동없음.csv


📁 파일 병합 중:  43%|███████████████████████████▊                                     | 3/7 [01:45<02:20, 35.19s/file]

✅ 중간 저장 완료: 병합/merged_step_test.csv (열 545)

📄 병합 중: test_5.잔액정보_EDA.csv


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 63: invalid start byte